# cellhashR HTO demultiplexing

In [ ]:
library(cellhashR)

CallAndGenerateReport(rawCountData = "G:/Belkaid_Lab_RNASeq_Data/Claudia_Rivera/single_cell_data/DC_20231026/scRNA_seq_gut_aggr_normalized4/outs/count/filtered_feature_bc_matrix", reportFile = 'report.html', callFile = 'calls.txt', title = 'Cell Hashing For Experiment DC_Claudia', methods = c('bff_cluster', 'htodemux', 'bff_raw', 'multiseq', 'dropletutils'), majorityConsensusThreshold = 0.59, doTSNE = FALSE,datatypeName = 'Antibody Capture')


In [ ]:
calls <- read.csv('calls.txt', sep='\t')

In [ ]:
# library(cellhashR)
# # Example 1: parse CITE-seq-Count output, printing QC
# barcodeData <- ProcessCountMatrix(rawCountData = "G:/Belkaid_Lab_RNASeq_Data/Claudia_Rivera/single_cell_data/DC_20231026/scRNA_seq_gut_aggr_normalized4/outs/count/filtered_feature_bc_matrix", minCountPerCell = 5, datatypeName='Antibody Capture')
# # Create QC plots of barcode normalization
# PlotNormalizationQC(barcodeData)

# # Generate the final cell hashing calls
# calls <- GenerateCellHashingCalls(barcodeMatrix = barcodeData,methods = c('bff_cluster', 'htodemux', 'bff_raw', 'multiseq', 'dropletutils'), chemistry='10xV2', metricsFile='methods_QC.html', majorityConsensusThreshold=0.59, doTSNE=FALSE) 

# Inspect negative cells:
# SummarizeCellsByClassification(calls = calls, barcodeMatrix = barcodeData)
# write.table(calls, file = 'calls.tsv', sep = '\t', row.names = FALSE, quote = FALSE)

### Add classification to seurat

In [ ]:
rownames(calls) <- calls$cellbarcode
scRNAseq <- AddMetaData(object = scRNAseq,metadata = calls['consensuscall'],col.name = 'consensuscall')
scRNAseq <- AddMetaData(object = scRNAseq,metadata = calls['consensuscall.global'],col.name = 'consensuscall.global')

### Data Normalization

In [ ]:
# Normalize RNA data with log normalization
scRNAseq <- NormalizeData(scRNAseq, assay = "RNA",normalization.method = "LogNormalize", scale.factor = 10000)

# Normalize HTO data with CLR
scRNAseq <- NormalizeData(scRNAseq, assay = "HTO",normalization.method = "CLR")

### tSNE visualization

In [ ]:
#tSNE-visualization
Idents(scRNAseq) <- "consensuscall.global"
unique(scRNAseq$consensuscall)
scRNAseq_subset <- subset(scRNAseq, idents = "Negative", invert =TRUE)
DefaultAssay(scRNAseq_subset) <- "HTO"
scRNAseq_subset <- ScaleData(scRNAseq_subset, assay = "HTO",  features = rownames(scRNAseq_subset), verbose = FALSE )
scRNAseq_subset <- RunPCA(scRNAseq_subset, assay = "HTO", rownames(scRNAseq_subset), approx = FALSE)
scRNAseq_subset <- RunTSNE(scRNAseq_subset, assay = "HTO", dims = 1:24, perplexity = 100, check_duplicates=FALSE)
DimPlot(scRNAseq_subset)
DimPlot_scCustom(scRNAseq_subset, group.by = 'consensuscall', pt.size=0.1)

### Filtering singlets

In [ ]:
# Filtering singlets only
Idents(scRNAseq) <- "consensuscall.global"
scRNAseq <-subset(scRNAseq, idents = "Singlet")